In [16]:
#Importing libraries and importing the csv into python and reading the first 5 transcations 
import pandas as pd
df = pd.read_csv("/Users/aarushsajeev/Desktop/Retail.csv", parse_dates=['date'])
df.head(5)

/var/folders/1n/rk03xkzj6hb1zddk5j99gv0h0000gn/T/ipykernel_36831/2596494218.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("/Users/aarushsajeev/Desktop/Retail.csv", parse_dates=['date'])


,date,invoice_id,customer_id,product,category,quantity,price,store,payment_type
0,2025-01-01,INV001,CUST001,Milk,Dairy,2,45,Main Branch,UPI
1,2025-01-01,INV001,CUST001,Bread,Bakery,1,35,Main Branch,UPI
2,2025-01-02,INV002,CUST002,shampoo,Personal,1,180,City Mall outlet,Credit Card
3,2025-01-02,INV002,CUST002,soap,Personal,3,40,City Mall outlet,Credit Card
4,2025-01-03,INV003,CUST003,Chips,Snacks,4,20,Main Branch,Cash


In [17]:
#Adding Total column into the transcations list
df['total'] = df['quantity'] * df['price']
df.head()

,date,invoice_id,customer_id,product,category,quantity,price,store,payment_type,total
0,2025-01-01,INV001,CUST001,Milk,Dairy,2,45,Main Branch,UPI,90
1,2025-01-01,INV001,CUST001,Bread,Bakery,1,35,Main Branch,UPI,35
2,2025-01-02,INV002,CUST002,shampoo,Personal,1,180,City Mall outlet,Credit Card,180
3,2025-01-02,INV002,CUST002,soap,Personal,3,40,City Mall outlet,Credit Card,120
4,2025-01-03,INV003,CUST003,Chips,Snacks,4,20,Main Branch,Cash,80


In [18]:
#Analysis for the table and the datatypes
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          10 non-null     datetime64[ns]
 1   invoice_id    10 non-null     object        
 2   customer_id   10 non-null     object        
 3   product       10 non-null     object        
 4   category      10 non-null     object        
 5   quantity      10 non-null     int64         
 6   price         10 non-null     int64         
 7   store         10 non-null     object        
 8   payment_type  10 non-null     object        
 9   total         10 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 932.0+ bytes


date            0
invoice_id      0
customer_id     0
product         0
category        0
quantity        0
price           0
store           0
payment_type    0
total           0
dtype: int64

In [19]:
#Graphical representation of daily reveune 
import plotly.express as px

daily = df.groupby('date')['total'].sum().reset_index()

px.line(daily, x='date', y='total', title='Daily Revenue Trend')


In [21]:
#top 10 products 
top_products = df.groupby('product')['total'].sum().sort_values(ascending=False).head(10)
top_products


product
shampoo    180
Pasta      120
soap       120
Cheese     110
Coke        90
Milk        90
Paneer      90
Chips       80
Butter      55
Bread       35
Name: total, dtype: int64

In [23]:
#Customer based basket value
basket = df.groupby('customer_id')['total'].sum().sort_values(ascending=False)
basket.head(10)


customer_id
CUST002    300
CUST005    230
CUST003    170
CUST004    145
CUST001    125
Name: total, dtype: int64

In [ ]:
#Revenue by weekdays
df['day'] = df['date'].dt.day_name()

weekday = df.groupby('day')['total'].sum()
weekday


day
Friday       170
Saturday     145
Sunday       230
Thursday     300
Wednesday    125
Name: total, dtype: int64

In [ ]:
#Recency, Frequency and Monetary of the customers
import datetime as dt

snapshot_date = df['date'].max() + pd.Timedelta(days=1)

rfm = df.groupby('customer_id').agg({
    'date': lambda x: (snapshot_date - x.max()).days,
    'invoice_id': 'nunique',
    'total': 'sum'
})

rfm.columns = ['Recency','Frequency','Monetary']
rfm.head()


,Recency,Frequency,Monetary
customer_id,,,
CUST001,5,1,125
CUST002,4,1,300
CUST003,3,1,170
CUST004,2,1,145
CUST005,1,1,230
